In [1]:
import csv
import itertools
import numpy as np
from scipy.spatial import distance_matrix

def get_dist_kaggle(file_path):
    coordinates = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile) 
        for row in reader:
            coordinates.append([float(row[0]), float(row[1])])
    coordinates = np.array(coordinates)
    dist_matrix = distance_matrix(coordinates, coordinates)
    return dist_matrix

def calcTourLength(hamPath, dist_matrix):
    tourLength=sum(dist_matrix[hamPath[0:-1], hamPath[1:len(hamPath)]])
    tourLength+=dist_matrix[hamPath[-1],hamPath[0]]
    return tourLength

def get_dist_tsplib(file_path):
    coordinates = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile) 
        inside_coords = False
        for row in reader:
            if row and row[0] == 'NODE_COORD_SECTION':  # Start reading coordinates
                inside_coords = True
                continue
            if row and row[0] == 'EOF':  # End of coordinates section
                inside_coords = False
                continue
            
            if inside_coords:
                # Extract coordinates (ignore the city number)
                x, y = row[0].split(" ")[1], row[0].split(" ")[2]
                coordinates.append((float(x), float(y)))
    dist_matrix = distance_matrix(coordinates, coordinates)
    return dist_matrix

def LK_heuristic(dist_matrix):
    """
    Implementation of Lin-Kernighan, an heuristic algorithm that solves the Traveling
    Salesman Problem using random swapping.

    Parameters:
        dists: distance matrix

    Returns:
        A tuple, (cost, path).
    """
    #Generate initial tour
    optlist = list(range(0, len(dist_matrix)))
    improvement=1

    while (improvement > 0):    #Check for every pair of cities that are neighbors in the tour whether improvement can be found
        bestTourLength = calcTourLength(optlist, dist_matrix)
        bestListSoFar = optlist
        improvement = -1
        
        for i in range(0, len(optlist)):
            #Given a pair of cities, find the swap that attains minimum distance with respect to current tour
            for j in range(2, len(optlist)-1):
                #Do a swap and see if tour length improves
                tempOptList = optlist[0:j]+optlist[:j-1:-1]
                tempTourLength = calcTourLength(tempOptList, dist_matrix)
                if(tempTourLength + 10e-12 < bestTourLength):
                    improvement = bestTourLength - tempTourLength
                    bestListSoFar = tempOptList
                    bestTourLength = tempTourLength
            if(bestTourLength+10e-12 < calcTourLength(optlist, dist_matrix)):
                optlist = bestListSoFar
                break
            optlist = [optlist[0]] + [optlist[-1]] + optlist[1:-1]

    # Calculate optimal cost
        opt = 0
        for k in range(len(optlist)-2):
            opt += dist_matrix[k, k+1]
            
    return opt, optlist

In [2]:
file_path = 'D:/Upenn/Numerical Analysis2/GroupProject/Dataset/tiny.csv'
dist_matrix = get_dist_kaggle(file_path)
opt, path = LK_heuristic(dist_matrix)
print(opt, path)

18.785708675082084 [0, 1, 2, 9, 8, 7, 6, 5, 4, 3]
